# Benchmarks

## Prep 

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score

from cnn.utils.graph_manipulation import load_frozen_graph, predict_from_frozen
from cnn.utils.dataset import load_cifar10, dataset_preprocessing_by_keras

In [2]:
pyperf = !find ~ -iname pyperf 2>/dev/null
pyperf = pyperf[0]

In [3]:
nets = ["dense", "conv", "medium"]
names = [[n + "_opt", n + "_quant"] for n in nets]
frozen_nets = [["models/" + n + ".pb"  for n in both] for both in names]
results_files = [["results/" + n + ".json"  for n in both] for both in names]

## Check accuracy 

In [4]:
_, _, x_test, t_test = load_cifar10()
x_test = dataset_preprocessing_by_keras(x_test)

In [43]:
for pb_files in frozen_nets:
    for m in pb_files:
        graph = load_frozen_graph(m)
        out = predict_from_frozen(graph, [x_test], ["features"], ["classes:0", "softmax:0"])
        classes = np.concatenate([batch[0] for batch in out])
        acc = accuracy_score(np.argmax(t_test, axis=1), classes)
        print(f"{m:22}: {acc:6} accuracy")

models/dense_opt.pb   : 0.6358 accuracy
models/dense_quant.pb :  0.622 accuracy
models/conv_opt.pb    : 0.7345 accuracy
models/conv_quant.pb  : 0.7348 accuracy
models/medium_opt.pb  : 0.6461 accuracy
models/medium_quant.pb: 0.6476 accuracy


## Time comparisons 

In [ ]:
for r in results_files:
    !{pyperf} show {r[0]} --quiet

In [ ]:
for r in results_files:
    !{pyperf} compare_to {r[0]} {r[1]} --table